### Loading Libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPool2D, Dropout, Flatten
from sklearn.metrics import accuracy_score,precision_score,confusion_matrix
from sklearn.preprocessing import normalize


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import cv2
import glob 
import math 
import shutil
import os
import time
import json
plt.rcParams['figure.figsize'] = [15, 5]
base_log_dir = "./logs/"
sns.set_style("darkgrid")

if not os.path.exists(base_log_dir):
    os.makedirs(base_log_dir)

### Plotting utils

In [ ]:
def CalculateMetricsAndPlot(true_label, predicted_label,color="Blues",text=""):
    plt.rcParams['figure.figsize'] = [9, 6]
    CM = confusion_matrix(true_label, predicted_label)
    acc = round(accuracy_score(true_label,predicted_label)*100,2)
    precision = round(precision_score(true_label,predicted_label, average='macro'),2)
    if text == "":
        sns.heatmap(CM ,annot=True, cmap=color, fmt='g').set_title("Confusion Matrix for Test Data | Accuracy={0}% | Precision={1}".format(acc,precision))
    else :
        sns.heatmap(CM ,annot=True, cmap=color, fmt='g').set_title("Confusion Matrix for Test Data | Accuracy={0}% | Precision={1} | {2}".format(acc,precision,text))
    
    plt.show()

def TrainingPlot(_history,text=None):
    plt.rcParams['figure.figsize'] = [15, 5]
    plt.subplot(1,2,1)
    plt.plot(_history["loss"],label="Train loss")
    plt.plot(_history["val_loss"],label="Validation loss")
    if not text is None:
        plt.title("Loss/Epoch | {0}".format(text))
    else:
        plt.title("Loss/Epoch")
    plt.xlabel("Epoch")
    plt.ylabel("Sparse Categorical Crossentropy")
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(_history["accuracy"],label="Train accuracy")
    plt.plot(_history["val_accuracy"],label="Validation accuracy")
    if not text is None:
        plt.title("Accuracy/Epoch | {0}".format(text))
    else:
        plt.title("Accuracy/Epoch")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy in %")
    plt.legend()
  
    plt.tight_layout()
    plt.show()


### Loading Dataset

In [ ]:
import tensorflow_datasets as tfds

(X_train, Y_train), (X_test, Y_test) , (X_valid, Y_valid) =  tfds.as_numpy(tfds.load('beans', 
    split = ['train', 'test','validation'], 
    batch_size=-1, 
    as_supervised=True))


### Preprocessing on dadtaset

In [ ]:
def ConvertDataset(Size,Chnl):
    _X_train = np.zeros((X_train.shape[0],Size,Size,Chnl))
    _X_test = np.zeros((X_test.shape[0],Size,Size,Chnl))
    _X_valid = np.zeros((X_valid.shape[0],Size,Size,Chnl))

    for index in range(X_train.shape[0]):
        tmp = cv2.resize(np.array(X_train[index]),(Size,Size),interpolation=cv2.INTER_AREA)
        if Chnl == 1:
          tmp = cv2.cvtColor(tmp,cv2.COLOR_RGB2GRAY)
        _X_train[index,:,:,:] = np.reshape(tmp,(Size,Size,Chnl))

    for index in range(X_test.shape[0]):
        tmp = cv2.resize(np.array(X_test[index]),(Size,Size),interpolation=cv2.INTER_AREA)
        if Chnl == 1:
          tmp = cv2.cvtColor(tmp,cv2.COLOR_RGB2GRAY)
        _X_test[index,:,:,:] = np.reshape(tmp,(Size,Size,Chnl))

    for index in range(X_valid.shape[0]):
        tmp = cv2.resize(np.array(X_valid[index]),(Size,Size),interpolation=cv2.INTER_AREA)
        if Chnl == 1:
          tmp = cv2.cvtColor(tmp,cv2.COLOR_RGB2GRAY)
        _X_valid[index,:,:,:] = np.reshape(tmp,(Size,Size,Chnl))

    return _X_train,_X_test, _X_valid

### Functions to make primary and custom LeNet

In [ ]:
def LeNetMaker(input_size=(32,32,1)):
    model = Sequential()
    model.add(keras.layers.InputLayer(input_size))
    model.add(keras.layers.Rescaling(1./255))
    model.add(keras.layers.Conv2D(6,kernel_size=(5,5),strides=(1,1),activation="tanh", padding='valid'))
    model.add(keras.layers.AveragePooling2D((2,2),strides=2))

    model.add(keras.layers.Conv2D(16,kernel_size=(5,5),strides=(1,1),activation="tanh", padding='valid'))
    model.add(keras.layers.AveragePooling2D((2,2),strides=2))

    model.add(keras.layers.Conv2D(120,kernel_size=(5,5),strides=(1,1),activation="tanh", padding='valid'))

    model.add(keras.layers.Flatten())

    model.add(keras.layers.Dense(84,activation="tanh"))

    model.add(keras.layers.Dense(3,activation="softmax"))

    return model 

def LeNetMakerWithRelu(input_size=(32,32,1)):
    model = Sequential()
    model.add(keras.layers.InputLayer(input_size))
    model.add(keras.layers.Rescaling(1./255))
    model.add(keras.layers.Conv2D(6,kernel_size=(5,5),strides=(1,1),activation="relu", padding='valid'))
    model.add(keras.layers.AveragePooling2D((2,2),strides=2))

    model.add(keras.layers.Conv2D(16,kernel_size=(5,5),strides=(1,1),activation="relu", padding='valid'))
    model.add(keras.layers.AveragePooling2D((2,2),strides=2))

    model.add(keras.layers.Conv2D(120,kernel_size=(5,5),strides=(1,1),activation="relu", padding='valid'))

    model.add(keras.layers.Flatten())

    model.add(keras.layers.Dense(84,activation="relu"))

    model.add(keras.layers.Dense(3,activation="softmax"))

    return model 

def CustomLeNetMaker(input_size=(32,32,3),drop_out=None,regularization=None,C1_n=6,C1_s=5,C2_n=16,C2_s=5,C3_n=120,C3_s=5,fc=84,activation="tanh"):
    model = Sequential()
    model.add(keras.layers.InputLayer(input_size))

    if regularization is None :
      model.add(keras.layers.Conv2D(C1_n,kernel_size=(C1_s,C1_s),strides=(1,1),activation=activation, padding='valid'))
    else:
      model.add(keras.layers.Conv2D(C1_n,kernel_size=(C1_s,C1_s),strides=(1,1),activation=activation, padding='valid',kernel_regularizer=regularization))

    if not drop_out is None:
      model.add(keras.layers.Dropout(drop_out))

    model.add(keras.layers.AveragePooling2D((2,2),strides=2))   

    if regularization is None :
      model.add(keras.layers.Conv2D(C2_n,kernel_size=(C2_s,C2_s),strides=(1,1),activation=activation, padding='valid'))
    else:
      model.add(keras.layers.Conv2D(C2_n,kernel_size=(C2_s,C2_s),strides=(1,1),activation=activation, padding='valid',kernel_regularizer=regularization))
    
    if not drop_out is None:
      model.add(keras.layers.Dropout(drop_out))

    model.add(keras.layers.AveragePooling2D((2,2),strides=2))   

    if regularization is None :
      model.add(keras.layers.Conv2D(C3_n,kernel_size=(C3_s,C3_s),strides=(1,1),activation=activation, padding='valid'))
    else:
      model.add(keras.layers.Conv2D(C3_n,kernel_size=(C3_s,C3_s),strides=(1,1),activation=activation, padding='valid',kernel_regularizer=regularization))
    
    if not drop_out is None:
      model.add(keras.layers.Dropout(drop_out))


    model.add(keras.layers.Flatten())

    model.add(keras.layers.Dense(fc,activation=activation))

    model.add(keras.layers.Dense(3,activation="softmax"))

    return model 

# Main study

### Scenario A) Exactly LeNet-5 (32,32,1)

In [ ]:
_X_train,_X_test, _X_valid = ConvertDataset(32,1)
model = LeNetMaker((32,32,1))
epochs = 50
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
History = model.fit(_X_train, Y_train, epochs=epochs, validation_data=(_X_valid,Y_valid),shuffle=True,verbose=0)
CalculateMetricsAndPlot(Y_test,np.argmax(model.predict(_X_test),axis=1),text="Scenario A")
TrainingPlot(History.history,text="Scenario A")
model.summary()
keras.utils.plot_model(model,to_file="Scenario A.png",expand_nested="True",show_layer_activations=True,show_shapes=True,show_layer_names=False)



### Scenario B) Only-Input Changed to orginal Beans shape(500,500,3)

In [ ]:
model = LeNetMaker((500,500,3))
epochs = 20
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
History = model.fit(X_train, Y_train, epochs=epochs, validation_data=(X_valid,Y_valid),shuffle=True,verbose=0)
CalculateMetricsAndPlot(Y_test,np.argmax(model.predict(X_test),axis=1),text="Scenario B",color="Greens")
TrainingPlot(History.history,text="Scenario B")
model.summary()
keras.utils.plot_model(model,to_file="Scenario B.png",expand_nested="True",show_layer_activations=True,show_shapes=True,show_layer_names=False)


### Scenario C) Only-Activation function Changed (Relu)

In [ ]:
_X_train,_X_test, _X_valid = ConvertDataset(32,1)
model = LeNetMakerWithRelu((32,32,1))
epochs = 50
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
History = model.fit(_X_train, Y_train, epochs=epochs, validation_data=(_X_valid,Y_valid),shuffle=True,verbose=0)
CalculateMetricsAndPlot(Y_test,np.argmax(model.predict(_X_test),axis=1),text="Scenario C",color="Reds")
TrainingPlot(History.history,text="Scenario C")
model.summary()
keras.utils.plot_model(model,to_file="Scenario C.png",expand_nested="True",show_layer_activations=True,show_shapes=True,show_layer_names=False)


### Scenario D) Only-input channel changed to RGB (32,32,3)

In [ ]:
_X_train,_X_test, _X_valid = ConvertDataset(32,3)
model = LeNetMaker((32,32,3))
epochs = 50
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
History = model.fit(_X_train, Y_train, epochs=epochs, validation_data=(_X_valid,Y_valid),shuffle=True,verbose=0)
CalculateMetricsAndPlot(Y_test,np.argmax(model.predict(_X_test),axis=1),text="Scenario D",color="Oranges")
TrainingPlot(History.history,text="Scenario D")
model.summary()
keras.utils.plot_model(model,to_file="Scenario D.png",expand_nested="True",show_layer_activations=True,show_shapes=True,show_layer_names=False)


### Scenario E) Activation function Changed (Relu) and input is set colorful (32,32,3)

In [ ]:
_X_train,_X_test, _X_valid = ConvertDataset(32,3)
model = LeNetMakerWithRelu((32,32,3))
epochs = 50
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
History = model.fit(_X_train, Y_train, epochs=epochs, validation_data=(_X_valid,Y_valid),shuffle=True,verbose=0)
CalculateMetricsAndPlot(Y_test,np.argmax(model.predict(_X_test),axis=1),text="Scenario E",color="cool")
TrainingPlot(History.history,text="Scenario E")
model.summary()
keras.utils.plot_model(model,to_file="Scenario E.png",expand_nested="True",show_layer_activations=True,show_shapes=True,show_layer_names=False)


# Regularization Study

### Drop Out Study

In [ ]:
_X_train,_X_test, _X_valid = ConvertDataset(32,1)
for rate in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8] :
    Text = "Input: (32,32,1) | Drop Out: " + str(rate)
    model = CustomLeNetMaker((32,32,1),drop_out=rate)
    epochs = 50
    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    History = model.fit(_X_train, Y_train, epochs=epochs, validation_data=(_X_valid,Y_valid),shuffle=True,verbose=0)
    CalculateMetricsAndPlot(Y_test,np.argmax(model.predict(_X_test),axis=1),text=Text,color="Blues")
    TrainingPlot(History.history,text=Text)

model.summary()
keras.utils.plot_model(model,to_file="Drop-out.png",expand_nested="True",show_layer_activations=True,show_shapes=True,show_layer_names=False)

### L1 regularize Study

In [ ]:
_X_train,_X_test, _X_valid = ConvertDataset(32,1)
for rate in [0.001,0.004,0.007,0.01] :
    Text = "Input: (32,32,1) | L1: " + str(rate)
    L1 = keras.regularizers.L1(rate)
    model = CustomLeNetMaker((32,32,1),regularization=L1)
    epochs = 50
    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    History = model.fit(_X_train, Y_train, epochs=epochs, validation_data=(_X_valid,Y_valid),shuffle=True,verbose=0)
    CalculateMetricsAndPlot(Y_test,np.argmax(model.predict(_X_test),axis=1),text=Text,color="Greens")
    TrainingPlot(History.history,text=Text)

model.summary()
keras.utils.plot_model(model,to_file="L1.png",expand_nested="True",show_layer_activations=True,show_shapes=True,show_layer_names=False)

### L2 regularize Study

In [ ]:
_X_train,_X_test, _X_valid = ConvertDataset(32,1)
for rate in [0.01,0.03,0.05,0.07,0.09,0.1] :
    Text = "Input: (32,32,1) | L2: " + str(rate)
    L2 = keras.regularizers.L2(rate)
    model = CustomLeNetMaker((32,32,1),regularization=L2)
    epochs = 50
    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    History = model.fit(_X_train, Y_train, epochs=epochs, validation_data=(_X_valid,Y_valid),shuffle=True,verbose=0)
    CalculateMetricsAndPlot(Y_test,np.argmax(model.predict(_X_test),axis=1),text=Text,color="Reds")
    TrainingPlot(History.history,text=Text)

model.summary()
keras.utils.plot_model(model,to_file="L2.png",expand_nested="True",show_layer_activations=True,show_shapes=True,show_layer_names=False)

# Kernel Count Study

In [ ]:
_X_train,_X_test, _X_valid = ConvertDataset(32,1)
for rate in [0.25,0.5,0.75,1.25,1.5] :
    cn_1 = int(6*rate)+1
    cn_2 = int(16*rate)+1
    cn_3 = int(120*rate)+1
    Text = "Input: (32,32,1) | Kernel #: {0},{1},{2}".format(cn_1,cn_2,cn_3)
    model = CustomLeNetMaker((32,32,1),C1_n=cn_1,C2_n=cn_2,C3_n=cn_3)
    epochs = 50
    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    History = model.fit(_X_train, Y_train, epochs=epochs, validation_data=(_X_valid,Y_valid),shuffle=True,verbose=0)
    CalculateMetricsAndPlot(Y_test,np.argmax(model.predict(_X_test),axis=1),text=Text,color="Purples")
    TrainingPlot(History.history,text=Text)


    keras.utils.plot_model(model,to_file="Kernel Count.png",expand_nested="True",show_layer_activations=True,show_shapes=True,show_layer_names=False)

# Kernel Size Study

In [ ]:
_X_train,_X_test, _X_valid = ConvertDataset(32,1)
for rate in [1,2,3,4] :
    Text = "Input: (32,32,1) | Kernel size: {0}".format(rate)
    model = CustomLeNetMaker((32,32,1),C1_s=rate,C2_s=rate,C3_s=rate)
    epochs = 50
    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    History = model.fit(_X_train, Y_train, epochs=epochs, validation_data=(_X_valid,Y_valid),shuffle=True,verbose=0)
    CalculateMetricsAndPlot(Y_test,np.argmax(model.predict(_X_test),axis=1),text=Text,color="Greys")
    TrainingPlot(History.history,text=Text)


    keras.utils.plot_model(model,to_file="Kernel Size.png",expand_nested="True",show_layer_activations=True,show_shapes=True,show_layer_names=False)

# A Final Model

In [ ]:
_X_train,_X_test, _X_valid = ConvertDataset(32,3)
Text = "Input: (32,32,3)|Kernel size:3|L2:0.1|DropOut:0.15|ReLU|Kernel #:{0},{1},{2}".format(cn_1,cn_2,cn_3)
cn_1 = int(6*0.75)+1
cn_2 = int(16*0.75)+1
cn_3 = int(120*0.75)+1

L2 = keras.regularizers.L2(0.1)
model = CustomLeNetMaker((32,32,3),0.15,L2,C1_s=3,C2_s=3,C3_s=3,activation="relu",C1_n=cn_1,C2_n=cn_2,C3_n=cn_3)
epochs = 50
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
History = model.fit(_X_train, Y_train, epochs=epochs, validation_data=(_X_valid,Y_valid),shuffle=True,verbose=0)
CalculateMetricsAndPlot(Y_test,np.argmax(model.predict(_X_test),axis=1),text=Text,color="PuBu")
TrainingPlot(History.history,text=Text)
keras.utils.plot_model(model,to_file="final.png",expand_nested="True",show_layer_activations=True,show_shapes=True,show_layer_names=False)